In [14]:
import os
import cv2
import random
import pandas as pd
import numpy as np

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.svm import SVC
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from time import time
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

import glob

from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog

import albumentations as A
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)

Функция для вывода изображений и оценки их текущего состояния:

In [15]:
def show_images(images, labels, randomize=True, rows=3, columns=10):
    fig, axx = plt.subplots(rows, columns, figsize=(15, rows*2))
    for i in range(rows):
        for j in range(columns):
            if randomize:
                idx = random.randint(0, len(images)-1)
            else:
                idx = i * columns + j
            if idx >= len(images):
                break
            img = images[idx]
            title = labels[idx]
            axx[i][j].axis("off")
            axx[i][j].set_title(title)
            axx[i][j].imshow(img, cmap='bone')

Перед началом работы с изображениями отметим, что тренировочный датасет довольно маленький по сравнению с тестовым.

---



Сравнив фотографии в датасете для обучения и тестирования в соревновании можно выявить, фотографии каких людей оказались отобраны. Далее можно увеличить датасет для обучения с помощью фотографий этих людей в публичном доступе (из хранилища для сбора фотографий для соревнования).

При считывании будем проводить предварительную обработку фотографий: переведём их в черно-белый и обрежем края, чтобы убрать лишние детали.

In [16]:
labels_map = {i: '' for i in range(10)}
labels_map[0] = 'Ирина'
labels_map[1] = 'Ruslanmgm'
labels_map[2] = 'eremenkokath'
labels_map[3] = 'Joe Duplantier'
labels_map[4] = 'Aleksandr V'
labels_map[5] = 'Kate_A'
labels_map[6] = 'Matveev_Denis'
labels_map[7] = 'Alexeeva'
labels_map[8] = 'Denis G'
labels_map[9] = 'Johnny Depp'
dirname = "Start_ML"
save_dirname = "Start_ML_processed_data/ML_track_train"


for cls, name in tqdm(labels_map.items()):
    subdir = os.path.join(dirname, name)
    filelist = os.listdir(subdir)

    for i, fname in tqdm(enumerate(filelist)):
        img = cv2.imdecode(np.fromfile(os.path.join(subdir, fname),
                                       dtype=np.uint8),
                           cv2.IMREAD_GRAYSCALE)
        h, w = img.shape
        size = min(h, w)
        h0 = int((h - size) / 2)
        w0 = int((w - size) / 2)
        img = img[h0: h0 + size, w0: w0 + size]

        img = cv2.resize(img, (64, 64), interpolation = cv2.INTER_AREA)
        cv2.imwrite(os.path.join(save_dirname, f'img{i}_label{cls}.png'), img)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  9.59it/s]
2it [00:00,  9.34it/s]
3it [00:00,  9.40it/s]
4it [00:00,  9.62it/s]
5it [00:00,  9.66it/s]
6it [00:00,  9.40it/s]
7it [00:00,  9.27it/s]
9it [00:00,  9.39it/s]
10it [00:01,  9.17it/s]
12it [00:01,  9.40it/s]
13it [00:01,  9.39it/s]
14it [00:01,  9.43it/s]
16it [00:01,  9.50it/s]
17it [00:01,  9.54it/s]
18it [00:01,  9.47it/s]
19it [00:02,  9.40it/s]
20it [00:02,  9.49it/s]
22it [00:02,  9.50it/s]
23it [00:02,  9.30it/s]
24it [00:02,  9.37it/s]
25it [00:02,  9.28it/s]
27it [00:02,  9.67it/s]
28it [00:02,  9.49it/s]
29it [00:03,  9.38it/s]
30it [00:03,  9.27it/s]
31it [00:03,  9.45it/s]
33it [00:03,  9.38it/s]
34it [00:03,  9.45it/s]
35it [00:03,  9.36it/s]
36it [00:03,  9.32it/s]
38it [00:04,  9.52it/s]
39it [00:04,  9.55it/s]
40it [00:04,  9.45it/s]
42it [00:04,  9.50it/s]
44it [00:04,  9.63it/s]
45it [00:04,  9.32it/s]
46it 

In [17]:
folder = "Start_ML_processed_data/"
test_data = os.path.join(folder + 'ML_track_test')
train_data = os.path.join(folder + 'ML_track_train')

In [157]:
train_x, train_y = [], []
test_x, test_y = [], []

for fname in tqdm(os.listdir(train_data)):
    if not('.jpg' in fname):
      img = cv2.imread(os.path.join(train_data, fname), cv2.IMREAD_GRAYSCALE)
      train_x.append(img)
      train_y.append(int(fname.split('label')[1].split('.png')[0]))


def img_number(fname):
    return int(fname.split('.')[0].split('image')[1])


for fname in tqdm(sorted(os.listdir(test_data), key=img_number)):
    img = cv2.imread(os.path.join(test_data, fname), cv2.IMREAD_GRAYSCALE)
    test_x.append(img)
    test_y.append(fname)

100%|████████████████████████████████████████████████████████████████████████████| 2090/2090 [00:00<00:00, 7530.79it/s]


In [1]:
# Показ фотографий убран при выкладывании ноутбука в публичное пространство
# show_images(train_x, train_y, randomize=False)

Воспользуемся аугментациями для ещё большего расширения датасета для обучения:

(кроме стандартных есть и Гауссовский шум, так как в тестовом датасете присутствуют зашумленные фотографии)

In [158]:
def augment_image(image):
    angle = np.arange(-10, 11, 1)
    angle0 = random.choice(angle)

    shift = 0.01 * np.arange(-10, 11, 1)
    shift0 = random.choice(shift)

    transform = A.Compose([
                           A.HorizontalFlip(p=0.4),
                           A.GaussNoise(var_limit=(75, 100), p=0.3),
                           # A.PiecewiseAffine... requires another version of A
                           # A.PiecewiseAffine(p=0.3),
                           A.RandomBrightnessContrast(brightness_limit=0.3,
                                                      contrast_limit=0.3,
                                                      p=0.3),
                           A.OneOf([
                                    A.MotionBlur(p=.2),
                                    A.MedianBlur(blur_limit=3, p=0.5),
                                    A.Blur(blur_limit=3, p=0.5),
                           ], p=0.2),
                           A.ShiftScaleRotate(shift_limit=0.0625,
                                              scale_limit=0, rotate_limit=13,
                                              p=0.2)
                          ])

    return transform(image=image)['image']

def get_augments(images, labels, augments_amount=20):
    x, y = [], []
    for image, label in tqdm(zip(images, labels)):
        for i in range(augments_amount):
            x.append(augment_image(image))
            y.append(label)
    return x, y

In [ ]:
# Показ фотографий убран при выкладывании ноутбука в публичное пространство
# show_images(train_x, train_y, randomize=False)

Воспользуемся каскадами Хаара для выделения лиц (для такого небольшого размера фотографий он показал себя лучше DLIB CNN):

In [159]:
def haar_application(images, labels):
    cascade_versions = ['haarcascade_frontalface_default.xml',
                        'haarcascade_frontalface_alt.xml',
                        'haarcascade_frontalface_alt2.xml']
    faces, labels_with_faces, no_faces, labels_without_faces = [], [], [], []
    haar_params = {'maxSize': (64, 64),
                   'scaleFactor': 1.01,
                   'minNeighbors': 2,
                   'minSize': (1, 1)}

    for _, (img, label) in tqdm(enumerate(zip(images, labels))):

        for version in cascade_versions:
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + version)
            face = face_cascade.detectMultiScale(img, **haar_params)
            if len(face) != 0:
               break

        for (a, b, w, h) in face:
            face = img[b:b + h, a:a + w]

        try:
            h, w = face.shape
            size = min(h, w)
            h0 = int((h - size) / 2)
            w0 = int((w - size) / 2)

            face = face[h0:h0 + size, w0:w0 + size]
            face = cv2.resize(face, (64, 64), interpolation=cv2.INTER_AREA)
            faces.append(face)
            labels_with_faces.append(label)

        except:
            no_faces.append(img)
            labels_without_faces.append(label)

    return faces, labels_with_faces, no_faces, labels_without_faces

В ходе экспериментов выяснилось, что точность предсказаний повышается при детекции лиц на предварительно аугментированных фотографиях (в отличие от обратной последовательности "детекция лиц --> аугментация").

In [160]:
train_x, train_y = get_augments(train_x, train_y)

train_faces, train_faces_labels, train_no_faces, train_no_faces_labels =\
  haar_application(train_x, train_y)

test_faces, test_faces_labels, test_no_faces, test_no_faces_labels =\
  haar_application(test_x, test_y)

516it [00:01, 355.79it/s]
10320it [05:38, 30.50it/s]
2090it [01:07, 31.08it/s]


In [ ]:
# Показ фотографий убран при выкладывании ноутбука в публичное пространство
# show_images(train_faces, train_faces_labels, randomize=False)

In [ ]:
# Показ фотографий убран при выкладывании ноутбука в публичное пространство
# show_images(train_no_faces, train_no_faces_labels, randomize=False)

In [161]:
for img, label in tqdm(zip(train_faces, train_faces_labels)):
  cv2.imwrite(os.path.join(save_dirname + "train_faces", str(label) + ".png"), img)
for img, label in tqdm(zip(train_no_faces, train_no_faces_labels)):
  cv2.imwrite(os.path.join(save_dirname + "train_no_faces", str(label) + ".png"), img)

for img, label in tqdm(zip(test_faces, test_faces_labels)):
  cv2.imwrite(os.path.join(save_dirname + "test_faces", str(label) + ".png"), img)
for img, label in tqdm(zip(test_no_faces, test_no_faces_labels)):
  cv2.imwrite(os.path.join(save_dirname + "test_no_faces", str(label) + ".png"), img)

9372it [00:00, 29340.36it/s]
948it [00:00, 23714.86it/s]
1896it [00:00, 24466.29it/s]
194it [00:00, 275362.09it/s]


И выявим признаки с помощью гистограммы направленных градиентов (в данном случае показала себя лучше SIFT):

In [162]:
def apply_hog_to_resized(images):
  hog = cv2.HOGDescriptor()
  hog_images = []
  for image in images:
      image = cv2.resize(np.uint8(image), (64, 128),
                         interpolation = cv2.INTER_AREA)
      hog_images.append(hog.compute(image, ))
  return hog_images


train_faces, train_no_faces, test_faces, test_no_faces =\
 map(lambda x: apply_hog_to_resized(x),
     [train_faces, train_no_faces, test_faces, test_no_faces])

In [163]:
train_faces, train_no_faces, test_faces, test_no_faces =\
 map(lambda x: np.asarray([el.ravel() for el in x]),
     [train_faces, train_no_faces, test_faces, test_no_faces])

Итоговое распределение классов:

In [164]:
from collections import Counter
print(Counter(train_no_faces_labels))
print(Counter(train_faces_labels))

Counter({1: 413, 6: 379, 9: 52, 7: 48, 0: 41, 5: 15})
Counter({2: 1560, 8: 1060, 3: 1000, 0: 999, 5: 985, 7: 972, 4: 900, 9: 688, 6: 621, 1: 587})


# Проверка качества моделей:

Делим тренировачный датасет на две выборки (для обучения и проверки):

In [166]:
x_educate, x_check, y_educate, y_check =\
 train_test_split(train_faces, train_faces_labels,
                  stratify=train_faces_labels, test_size=0.3)
x_nf_educate, x_nf_check, y_nf_educate, y_nf_check =\
 train_test_split(train_no_faces, train_no_faces_labels,
                  stratify=train_no_faces_labels, test_size=0.3)

Берём 2 модели: для изображений с выявленными лицами и без.

In [167]:
pca = KernelPCA()
model_svm = SVC(probability=True)

first_model = make_pipeline(PCA(150), SVC(class_weight='balanced', C=150, gamma=0.05, verbose=2))
first_model.fit(x_educate, y_educate)

second_model = make_pipeline(PCA(150), SVC(class_weight='balanced', C=150, gamma=0.05, verbose=2))
second_model.fit(x_nf_educate, y_nf_educate)

[LibSVM][LibSVM]

Pipeline(steps=[('pca', PCA(n_components=150)),
                ('svc',
                 SVC(C=150, class_weight='balanced', gamma=0.05, verbose=2))])

In [169]:
preds_1 = first_model.predict(x_check)
preds_2 = second_model.predict(x_nf_check)

In [170]:
print('FIRST_MODEL: ', accuracy_score(y_check, preds_1))
print('SECOND MODEL: ', accuracy_score(y_nf_check, preds_2))

FIRST_MODEL:  0.9893314366998578
SECOND MODEL:  0.9649122807017544


Возникает стойкое ощущение, что случается переобучение.
Private score: 0.82296\
  (довольно низкий по сравнению с полученным выше, можно подтвердить наличие переобучения).

---



Для улучшения результата можно попробовать другие модели. Например, решающие деревья. Чтобы не переобучаться, будем использовать ExtraTreesClassifier. Все параметры перебирались по сетке (для предыдущей модели аналогично).

In [171]:
def grid_search(models: list, grid_params, cross_val, verbose=3,
                        scoring='accuracy', n_jobs=3, refit=True):
    pipeline = make_pipeline(*models)
    search = GridSearchCV(pipeline, grid_params, cv=cross_val, scoring=scoring,
                          verbose=verbose, n_jobs=n_jobs, refit=refit)
    return search

cv = KFold(n_splits=3, shuffle=True, random_state=0)
models = [PCA(), ExtraTreesClassifier(class_weight='balanced', n_jobs=4,
                                      verbose=2)]
grid = {
    'pca__n_components': [100, 150, 200, 300, 400],
    'extratreesclassifier__n_estimators': [10, 20, 50, 100, 200],
    'extratreesclassifier__criterion': ['gini'],
    'extratreesclassifier__max_depth': [5, 20, None],
    'extratreesclassifier__min_samples_split': [2, 10, 0.1],
    'extratreesclassifier__min_samples_leaf': [1, 20, 50],
    'extratreesclassifier__max_features': ['auto', 'log2']
    }

pipe = grid_search(models, grid, cv, n_jobs=1)


Подбираем наилучшие гиперпараметры:

In [ ]:
def get_best_params(pipe, X, y):
    start_time = time()
    pipe.fit(X, y)
    print("done in %0.3fs" % (time() - start_time))
    print("Best parameter (CV score=%0.3f):" % pipe.best_score_)
    print(pipe.best_params_)
    return pipe

pipe = get_best_params(pipe, x_educate, y_educate)

Fitting 3 folds for each of 1350 candidates, totalling 4050 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    9.6s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


[CV 1/3] END extratreesclassifier__criterion=gini, extratreesclassifier__max_depth=5, extratreesclassifier__max_features=auto, extratreesclassifier__min_samples_leaf=1, extratreesclassifier__min_samples_split=2, extratreesclassifier__n_estimators=10, pca__n_components=100;, score=0.669 total time=  10.0s


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


[CV 2/3] END extratreesclassifier__criterion=gini, extratreesclassifier__max_depth=5, extratreesclassifier__max_features=auto, extratreesclassifier__min_samples_leaf=1, extratreesclassifier__min_samples_split=2, extratreesclassifier__n_estimators=10, pca__n_components=100;, score=0.633 total time=   0.3s


...

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished


[CV 3/3] END extratreesclassifier__criterion=gini, extratreesclassifier__max_depth=5, extratreesclassifier__max_features=auto, extratreesclassifier__min_samples_leaf=1, extratreesclassifier__min_samples_split=0.1, extratreesclassifier__n_estimators=50, pca__n_components=100;, score=0.779 total time=   0.3s


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished


[CV 1/3] END extratreesclassifier__criterion=gini, extratreesclassifier__max_depth=5, extratreesclassifier__max_features=auto, extratreesclassifier__min_samples_leaf=1, extratreesclassifier__min_samples_split=0.1, extratreesclassifier__n_estimators=50, pca__n_components=150;, score=0.783 total time=   0.4s


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished


Переходим к самому обучению модели и проверки результатов:

In [172]:
trees_params = {
    'criterion': 'gini',
    'max_depth': 20,
    'max_features': 'log2',
    'min_samples_leaf': 1,
    'min_samples_split': 10,
    'n_estimators': 200
    }
first_model = make_pipeline(PCA(100), ExtraTreesClassifier(class_weight='balanced',
                                                           n_jobs=4, verbose=1,
                                                           **trees_params))
second_model = make_pipeline(PCA(100), ExtraTreesClassifier(class_weight='balanced',
                                                            n_jobs=4, verbose=1,
                                                            **trees_params))


first_model.fit(x_educate, y_educate)
second_model.fit(x_nf_educate, y_nf_educate)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.0s finished


Pipeline(steps=[('pca', PCA(n_components=100)),
                ('extratreesclassifier',
                 ExtraTreesClassifier(class_weight='balanced', max_depth=20,
                                      max_features='log2', min_samples_split=10,
                                      n_estimators=200, n_jobs=4, verbose=1))])

In [173]:
preds_1 = first_model.predict(x_check)
preds_2 = second_model.predict(x_nf_check)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.0s finished


In [174]:
print('FIRST_MODEL: ', accuracy_score(y_check, preds_1))
print('SECOND MODEL: ', accuracy_score(y_nf_check, preds_2))

FIRST_MODEL:  0.9715504978662873
SECOND MODEL:  0.9543859649122807


Private score: 0.89569



---



И проверим StackingClassifier:

С StandardScaler:

In [175]:
def get_stacked_estimator():
    estimators = [('forest', RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_split=10, n_jobs=4, verbose=0)),
                ('svm', SVC(class_weight='balanced', C=100, gamma='scale'))]
    final_clf = SVC(C=50)
    stacked = StackingClassifier(estimators=estimators, final_estimator=final_clf)
    return final_clf

first_model = make_pipeline(preprocessing.StandardScaler(), PCA(120),
                            get_stacked_estimator())
second_model = make_pipeline(preprocessing.StandardScaler(), PCA(120),
                             get_stacked_estimator())

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=120)), ('svc', SVC(C=50))])

In [176]:
first_model.fit(x_educate, y_educate)
second_model.fit(x_nf_educate, y_nf_educate)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=120)), ('svc', SVC(C=50))])

In [177]:
preds_1 = first_model.predict(x_check)
preds_2 = second_model.predict(x_nf_check)

In [178]:
print('FIRST_MODEL: ', accuracy_score(y_check, preds_1))
print('SECOND MODEL: ', accuracy_score(y_nf_check, preds_2))

FIRST_MODEL:  0.9861308677098151
SECOND MODEL:  0.968421052631579


Private score: 0.96937

---

C MinMaxScaler:

In [179]:
first_model = make_pipeline(preprocessing.MinMaxScaler (), PCA(120),
                            get_stacked_estimator())
second_model = make_pipeline(preprocessing.MinMaxScaler (), PCA(120),
                             get_stacked_estimator())

In [180]:
first_model.fit(x_educate, y_educate)
second_model.fit(x_nf_educate, y_nf_educate)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('pca', PCA(n_components=120)), ('svc', SVC(C=50))])

In [181]:
preds_1 = first_model.predict(x_check)
preds_2 = second_model.predict(x_nf_check)

In [182]:
print('FIRST_MODEL: ', accuracy_score(y_check, preds_1))
print('SECOND MODEL: ', accuracy_score(y_nf_check, preds_2))

FIRST_MODEL:  0.9882645803698435
SECOND MODEL:  0.9649122807017544


Private score: 0.96459

---

# Итог

В качестве оцениваемых результатов были выбраны предсказания от StackingClassifier с StandardScaler и MinMaxScaler.

In [183]:
first_model.fit(train_faces, train_faces_labels)
second_model.fit(train_no_faces, train_no_faces_labels)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('pca', PCA(n_components=120)), ('svc', SVC(C=50))])

In [184]:
test_preds_1 = first_model.predict(test_faces)
test_preds_2 = second_model.predict(test_no_faces)

In [185]:
preds = np.concatenate([test_preds_1, test_preds_2])
labels = np.concatenate([test_faces_labels, test_no_faces_labels])
pairs = sorted(zip(labels, preds), key=lambda x: int(x[0].split('.')[0].split('image')[1]))
pred_df = pd.DataFrame(pairs, columns = ['img_name', 'pred_class'])
pred_df.to_csv("submission.csv", index=False)
pred_df.head()

,img_name,pred_class
0,image0.png,6
1,image1.png,3
2,image2.png,4
3,image3.png,1
4,image4.png,0


**Public score:**



*   StandardScaler -- Score: 0.9665
*   MinMaxScaler -- Score: 0.96842



Результаты предсказаний остальных модели получили public score хуже.